In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/ considere as seguintes tabelas SQL

## Dados
* Os dados podem ser encontrados nesta __[pasta](https://github.com/Necctares/MC536/tree/main/lab04/data)__:
* __[demographic-person.csv (Person)](https://github.com/Necctares/MC536/blob/main/lab04/data/demographic-person.csv)__
* __[medicantions-drug (Drug)](https://github.com/Necctares/MC536/blob/main/lab04/data/medications-drug.csv)__
* __[medications-use (DrugUse)](https://github.com/Necctares/MC536/blob/main/lab04/data/medications-use.csv)__

In [2]:
DROP Table IF EXISTS Person;
DROP Table IF EXISTS Drug;
DROP Table IF EXISTS DrugUse;

In [3]:
CREATE TABLE Person (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('https://raw.githubusercontent.com/Necctares/MC536/main/lab04/data/demographic-person.csv');

CREATE TABLE Drug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(100) NOT NULL,
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('https://raw.githubusercontent.com/Necctares/MC536/main/lab04/data/medications-drug.csv');

CREATE TABLE DrugUse (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId)
    REFERENCES Person(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Drug(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('https://raw.githubusercontent.com/Necctares/MC536/main/lab04/data/medications-use.csv');

In [4]:
SELECT * FROM Person;
SELECT * FROM Drug;
SELECT * FROM DrugUse

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT DR.Name, COUNT(*) FROM Drug DR, DrugUse DU WHERE DR.Code = DU.DrugCode GROUP BY DR.Name;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [6]:
CREATE VIEW UsoMed AS SELECT DrugCode, COUNT(*) numero_uso FROM DrugUse GROUP BY DrugCode;
CREATE VIEW MaxUso AS SELECT MAX(numero_uso) AS max_uso FROM UsoMed;
SELECT U.DrugCode FROM UsoMed U, MaxUso M WHERE U.numero_uso = M.max_uso;

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [73]:
SELECT DrugCode, COUNT(*) AS max FROM DrugUse GROUP BY DrugCode HAVING max IN (SELECT MAX(max_uso) FROM (SELECT COUNT(*) AS max_uso FROM DrugUse GROUP BY DrugCode));

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [74]:
DROP TABLE IF EXISTS MaxUso;
DROP TABLE IF EXISTS UsoMed;
CREATE VIEW UsoMed AS SELECT DrugCode, COUNT(*) numero_uso FROM DrugUse GROUP BY DrugCode;
CREATE VIEW MaxUso AS SELECT MAX(numero_uso) AS max_uso FROM UsoMed;
SELECT D.Name FROM UsoMed U, MaxUso M, Drug D WHERE U.numero_uso = M.max_uso AND D.Code = U.DrugCode;

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [85]:
SELECT AVG(Num_Med_Usados) FROM (SELECT U.PersonId, COUNT(U.PersonId) AS Num_Med_Usados FROM DrugUse U GROUP BY PersonId);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [87]:
DROP VIEW IF EXISTS UsoMedPessoa;
CREATE VIEW UsoMedPessoa AS SELECT U.PersonId, COUNT(U.PersonId) AS Num_Med_Usados FROM DrugUse U GROUP BY PersonId;
SELECT U.PersonId FROM UsoMedPessoa U WHERE U.Num_Med_Usados > (SELECT AVG(Num_Med_Usados) FROM (SELECT U.PersonId, COUNT(U.PersonId) AS Num_Med_Usados FROM DrugUse U GROUP BY PersonId));

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [93]:
DROP VIEW IF EXISTS UsoMedIdade;
DROP VIEW IF EXISTS UsoMedPessoa;
CREATE VIEW UsoMedPessoa AS SELECT U.PersonId, COUNT(U.PersonId) AS Num_Med_Usados, P.Age FROM DrugUse U, Person P WHERE U.PersonId = P.Id GROUP BY PersonId;
CREATE VIEW UsoMedIdade AS SELECT UMP.Age, SUM(UMP.Num_Med_Usados) AS Num_Med_Usados FROM UsoMedPessoa UMP GROUP BY UMP.Age;
SELECT * FROM UsoMedIdade ORDER BY Num_Med_Usados DESC;

# Navegando por Hierarquias

A seguir exercícios envolvendo navegação por hierarquias.

# Marcadores e Taxonomia

Considere o modelo para `Marcadores` e `Taxonomia`:

![UML](marcadores-taxonomia-uml.png)

![Relacional](marcadores-taxonomia-er.png)

Considere as sentenças SQL a seguir para montar as tabelas de `Marcadores` e `Taxonomia`:

In [94]:
DROP TABLE IF EXISTS Taxonomia;
DROP TABLE IF EXISTS Marcadores;

CREATE TABLE Taxonomia (
  Categoria VARCHAR(50) NOT NULL,
  Superior  VARCHAR(50),
  PRIMARY KEY(Categoria),
  FOREIGN KEY(Superior)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

CREATE TABLE Marcadores (
  Titulo VARCHAR(50) NOT NULL,
  Endereco VARCHAR(80) NOT NULL,
  Acessos INTEGER,
  Categoria VARCHAR(50) NOT NULL,
  PRIMARY KEY(Titulo),
  FOREIGN KEY(Categoria)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

As tabelas são preenchidas com as seguintes tuplas:

In [95]:
INSERT INTO Taxonomia VALUES ('Geral', NULL);
INSERT INTO Taxonomia VALUES ('Serviços', 'Geral');
INSERT INTO Taxonomia VALUES ('Acadêmico', 'Geral');
INSERT INTO Taxonomia VALUES ('Relacionamento', 'Geral');
INSERT INTO Taxonomia VALUES ('Busca', 'Serviços');
INSERT INTO Taxonomia VALUES ('Portal', 'Serviços');
INSERT INTO Taxonomia VALUES ('Mail', 'Serviços');
INSERT INTO Taxonomia VALUES ('Vendas', 'Serviços');
INSERT INTO Taxonomia VALUES ('Universidade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('CG', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Sociedade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Rede Social', 'Relacionamento');
INSERT INTO Taxonomia VALUES ('Marketplace', 'Vendas');
INSERT INTO Taxonomia VALUES ('Loja', 'Vendas');
INSERT INTO Taxonomia VALUES ('Notícias', 'Portal');
INSERT INTO Taxonomia VALUES ('Agregador', 'Portal');

INSERT INTO Marcadores VALUES ('Terra', 'http://www.terra.com.br', 295, 'Portal');
INSERT INTO Marcadores VALUES ('POVRay', 'http://www.povray.org', 2, 'CG');
INSERT INTO Marcadores VALUES ('SBC', 'http://www.sbc.org.br', 26, 'Sociedade');
INSERT INTO Marcadores VALUES ('Correios', 'http://www.correios.com.br', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('GMail', 'http://www.gmail.com', 296, 'Mail');
INSERT INTO Marcadores VALUES ('Google', 'http://www.google.com', 1590, 'Busca');
INSERT INTO Marcadores VALUES ('Yahoo', 'http://www.yahoo.com', 134, 'Serviços');
INSERT INTO Marcadores VALUES ('Orkut', 'http://www.orkut.com', 45, 'Rede Social');
INSERT INTO Marcadores VALUES ('iBahia', 'http://www.ibahia.com', 3, 'Portal');
INSERT INTO Marcadores VALUES ('Submarino', 'http://www.submarino.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Amazon', 'https://www.amazon.com.br', 410, 'Marketplace');
INSERT INTO Marcadores VALUES ('Americanas', 'https://www.americanas.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Mercado Livre', 'https://www.mercadolivre.com.br', 500, 'Marketplace');
INSERT INTO Marcadores VALUES ('G1', 'https://g1.globo.com', 1200, 'Notícias');
INSERT INTO Marcadores VALUES ('Folha', 'https://www.folha.uol.com.br', 850, 'Notícias');
INSERT INTO Marcadores VALUES ('Google News', 'https://news.google.com', 900, 'Agregador');
INSERT INTO Marcadores VALUES ('Flipboard', 'https://flipboard.com', 950, 'Agregador');

Conteúdo da tabela `Taxonomia`:

In [96]:
SELECT * FROM Taxonomia;

Conteúdo da tabela `Marcadores`:

In [97]:
SELECT * FROM Marcadores;

## Tarefa 1

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, sem considerar as categorias subordinadas.

In [98]:
SELECT * FROM Marcadores M WHERE M.Categoria = 'Serviços';

## Tarefa 2

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, considerando as categorias subordinadas.

In [102]:
SELECT * FROM Marcadores M WHERE M.Categoria IN (SELECT Categoria FROM Taxonomia T WHERE T.Superior = 'Serviços') OR M.Categoria = 'Serviços';